In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
N_centr = 5
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="../VTXalignment/input/"
file_names=["my-m_ee_Run14AuAu_87th_new_19865_1065runs.root"] 
file_names=["my-m_ee_Run14AuAu_104th_new_19928_489runs.root"]
#m_ee_Run14AuAu_103rd_new_19927_255runs m_ee_Run14AuAu_104th_new_19928_489runs
#m_ee_Run14AuAu_100th_new_19922_447runs m_ee_Run14AuAu_101st_new_19924_207runs
#m_ee_Run14AuAu_98th_new_19918_1064runs m_ee_Run14AuAu_99th_new_19920_1063runs
sim_file_path= "../sim/output/summer25/electron/"
dalitz_file_names=['piminus_50M_new_v8.root']# piminus_50M_v8 piminus_50M_new_v3
dalitz_file_names = ["dalitz_v02.root"]
ccbar_name = ['ccbar_v00.root']#dalitz_v00
sim_file_names = [dalitz_file_names[0], ccbar_name[0]]
legend_names = ["real","#pi^{-}","c#bar{c}"] 
hist_select_3D_names = ["DCA2_hist_0","DCA2_hist_1","DCA2_hist_2","DCA2_hist_3","DCA2_hist_4","sDCA2_hist_0","sDCA2_hist_1","sDCA2_hist_2","sDCA2_hist_3","sDCA2_hist_4",\
                        "DCA_2D_hist_0","DCA_2D_hist_1","DCA_2D_hist_2","DCA_2D_hist_3","DCA_2D_hist_4","sDCA_2D_hist_0","sDCA_2D_hist_1","sDCA_2D_hist_2","sDCA_2D_hist_3","sDCA_2D_hist_4"]

In [3]:
N_hists = len(hist_select_3D_names)
loc = True
Save_to_html = False

In [4]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

../VTXalignment/input/my-m_ee_Run14AuAu_104th_new_19928_489runs.root
../sim/output/summer25/electron/dalitz_v02.root
../sim/output/summer25/electron/ccbar_v00.root


In [9]:
conv_type = [1,25]
pt = [.45,0.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+10
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real_prodfjproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[0][type_loc].GetName()+f"dalitz_dfprojproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    conv_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"ccbar_dfprojproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(conv_proj,Lcolor=2, Msize=0)
    real_proj.Scale(1./(1+real_proj.Integral()))
    piminus_proj.Scale(1./(1+piminus_proj.Integral()))
    conv_proj.Scale(1./(1+conv_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Draw("HIST")
    piminus_proj.Draw("sameH")
    conv_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    #print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    #print(f"Standard deviation for dalitz_proj in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    #print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

In [13]:
conv_type = [1,25]
pt = [1.45,4.55]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+5
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"rsdeal_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dasdlitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    ccbar_proj = hists_sim[1][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"ccbar_dfprojproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(ccbar_proj,Lcolor=2, Msize=0)
    real_proj.Scale(1./(1+real_proj.Integral()))
    piminus_proj.Scale(1./(1+piminus_proj.Integral()))
    ccbar_proj.Scale(1./(1+ccbar_proj.Integral()))  
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Fit("gaus", "", "", -120, 120)
    real_proj.Draw("HIST")
    piminus_proj.Draw("sameH")
    #ccbar_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    print(f"Standard deviation for simul in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

Standard deviation for real_proj in centrality bin 0: 104.58785257933324
Standard deviation for simul in centrality bin 0: 103.66719473816153
beam width = 13.846719549280968
Standard deviation for real_proj in centrality bin 1: 104.11618437138974
Standard deviation for simul in centrality bin 1: 93.30349303806038
beam width = 46.2021431857207
Standard deviation for real_proj in centrality bin 2: 110.30593984982035
Standard deviation for simul in centrality bin 2: 102.48362494025939
beam width = 40.798369884793516
Standard deviation for real_proj in centrality bin 3: 107.17979958666136
Standard deviation for simul in centrality bin 3: 100.03927311508866
beam width = 38.466261503317334
Standard deviation for real_proj in centrality bin 4: 124.1224063064604
Standard deviation for simul in centrality bin 4: 113.12005333917037
beam width = 51.09036386491384


In [7]:
conv_type = [1,25]
pt = [1.45,4.55]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"fprojy{0}")
projz = hists_read[0][0].ProjectionZ(f"ffprojz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+0
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"rsdeadl_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dasddlitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    real_proj.Scale(1./(1+real_proj.Integral()))
    piminus_proj.Scale(1./(1+piminus_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Fit("gaus", "", "", -120, 120)
    real_proj.Draw("HIST")
    piminus_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    print(f"Standard deviation for simul in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

Standard deviation for real_proj in centrality bin 0: 104.21145192315795
Standard deviation for simul in centrality bin 0: 99.8329139434488
beam width = 29.890065331019382
Standard deviation for real_proj in centrality bin 1: 101.78899349282625
Standard deviation for simul in centrality bin 1: 88.97694732001247
beam width = 49.437860409754286
Standard deviation for real_proj in centrality bin 2: 108.27260416748783
Standard deviation for simul in centrality bin 2: 98.04842171348801
beam width = 45.928899537257756
Standard deviation for real_proj in centrality bin 3: 104.79864804747454
Standard deviation for simul in centrality bin 3: 91.48830180484796
beam width = 51.11406132800926
Standard deviation for real_proj in centrality bin 4: 122.38121531149454
Standard deviation for simul in centrality bin 4: 108.35467078874386
beam width = 56.88960519621598
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      103.263
NDf                      